### Load Data

In [1]:
import pandas as pd

In [2]:
links = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/links.csv')
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [3]:
movies = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/movies.csv')

In [4]:
ratings = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/ratings.csv')


In [5]:
tags = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/tags.csv')

In [6]:
ratings = ratings.drop('timestamp', axis=1)

In [7]:
ratings.shape

(100836, 3)

In [8]:
new = pd.merge(links, movies, on='movieId')

In [9]:
new_2 = pd.merge(new, ratings, on='movieId')

In [10]:
new_3 = pd.merge(new_2, tags, on='movieId')

### Merge Data Frames

In [11]:
# merge links and movie
#frames_1 = [links, movies]
#new = pd.concat(frames_1)


In [12]:
# merge new and ratings
#frames_2 = [new, ratings]
#new_2 = pd.concat(frames_2)

In [13]:
# Drop movieId and timestamp
#tags_2 = tags.drop(columns=['movieId', 'timestamp'])

In [14]:
# concat new_2 and tags
#frames = [new_2, tags_2]
#new_3 = pd.concat(frames)
#new_3 = pd.merge(new_2, tags)


In [15]:
new_3.shape

(233213, 10)

### Preprocessing

In [16]:
# Check for null values
new_3.isnull().sum()

movieId      0
imdbId       0
tmdbId       0
title        0
genres       0
userId_x     0
rating       0
userId_y     0
tag          0
timestamp    0
dtype: int64

In [17]:
new_3['movieId'].nunique()

1554

In [18]:
# replace | with , in the genres column
new_3['genres'] = new_3['genres'].str.replace('|',' , ')

In [19]:
#df = new_3.drop_duplicates(subset=['movieId'])

In [20]:
# convert df to a csv file
new_3.to_csv('/Users/gabrielwarner/Data-Science/Projects/phase_4/movies_2.csv')

### Explode genres column

In [21]:
# split genre column based on ,
new_3['new_genres'] = new_3['genres'].str.split(',', 10)

In [22]:
# explode new_genres
df = new_3.explode('new_genres')

In [23]:
# drop genres from df
df_2 = df.drop(columns=['genres'])

In [24]:
df_2.head()

,movieId,imdbId,tmdbId,title,userId_x,rating,userId_y,tag,timestamp,new_genres
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Adventure
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Animation
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Children
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Comedy
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Fantasy


### OHE title

In [25]:
one_hot = pd.get_dummies(df_2['title'])

In [26]:
df = df_2.drop('title',axis = 1)

In [27]:
df = pd.concat([df, one_hot], axis=1)

In [28]:
df.shape

(765271, 1563)

In [29]:
one_hot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 1554 entries, (500) Days of Summer (2009) to eXistenZ (1999)
dtypes: uint8(1554)
memory usage: 1.1 GB


### OHE tag

In [30]:
one_hot_tag = pd.get_dummies(df['tag'])

In [31]:
df = df.drop('tag',axis = 1)

In [32]:
df = pd.concat([df, one_hot_tag], axis=1)

In [33]:
one_hot_tag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 1584 entries, "artsy" to zombies
dtypes: uint8(1584)
memory usage: 1.1 GB


### OHE new_genres

In [34]:
one_hot_genre = pd.get_dummies(df['new_genres'])

In [35]:
df =df.drop('new_genres',axis = 1)

In [36]:
df = pd.concat([df, one_hot_genre], axis=1)

### Base Line Model

In [37]:
from sklearn.neighbors import NearestNeighbors

In [38]:
#knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
#knn.fit(df)

### Suprise

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 3209 entries, movieId to Western
dtypes: float64(2), int64(5), uint8(3202)
memory usage: 2.3 GB


### Change Data Frames to suprise DataFrame

In [40]:
import surprise

In [41]:
from surprise.model_selection import train_test_split
# from sklearn.model_selection import train_test_split

In [42]:
ratings.rating.describe()

count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [43]:
ratings.shape

(100836, 3)

In [44]:
reader = surprise.Reader(rating_scale = (4., 5.))

In [45]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [46]:
data = surprise.Dataset.load_from_df(ratings, reader)

In [47]:
df.head()

,movieId,imdbId,tmdbId,userId_x,rating,userId_y,timestamp,(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),...,Musical,Musical,Mystery,Mystery,Romance,Romance,Sci-Fi,Sci-Fi,Thriller,Western
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [49]:
# from sklearn.model_selection import train_test_split

In [50]:
trainset, testset = train_test_split(data, test_size=0.2)

In [51]:
#trainset.head()

### KNNS Basic (Best RMSE=.5)

In [52]:
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy

In [53]:
basic = knns.KNNBasic()
basic.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [54]:
predictions = basic.test(testset)

In [55]:
print(accuracy.rmse(predictions))

RMSE: 1.1528
1.152846268779916


In [56]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1538
1.1538175854355321


### Matrix Factorization

In [57]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

### Suprise with grid search

In [58]:
from sklearn.neighbors import KNeighborsClassifier

In [59]:
param_grid = {'n_factors':[20, 50, 75, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
               'reg_all': [0.4, 0.6]}

In [60]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors':[20, 50, 75, 100],'n_epochs': [5, 6, 7, 8, 9, 10, 11], 'lr_all': [0.002, .003, .004, 0.005],
               'reg_all': [.02, 0.4, .5, 0.6]}
gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
gs_model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   50.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 2240 out of 2240 | elapsed: 13.7min finished


In [61]:
svd = SVD(n_factors=100, n_epochs=10, lr_all=0.005, reg_all=0.4)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.1484
1.1484375101919158


### Predictions SVD

In [62]:
user_34_prediction = svd.predict('12', '85')
user_34_prediction

Prediction(uid='12', iid='85', r_ui=None, est=4.0, details={'was_impossible': False})

In [63]:
user_34_prediction[3]

4.0

In [64]:
def get_Iu(uid):
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
def get_Ui(iid):
    """
args:
iid: the raw id of the item
returns:
the number of users that have rated the item.
"""
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
df_predictions = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_predictions['Iu'] = df_predictions.uid.apply(get_Iu)
df_predictions['Ui'] = df_predictions.iid.apply(get_Ui)
df_predictions['err'] = abs(df_predictions.est - df_predictions.rui)

In [65]:
best_predictions = df_predictions.sort_values(by='err')[:10]
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
14109,483,88163,4.0,4.0,{'was_impossible': False},578,25,0.0
13162,200,30825,4.0,4.0,{'was_impossible': False},262,27,0.0
4720,561,2529,4.0,4.0,{'was_impossible': False},408,39,0.0
13166,525,551,4.0,4.0,{'was_impossible': False},400,70,0.0
13167,273,410,4.0,4.0,{'was_impossible': False},38,65,0.0
4715,184,174055,4.0,4.0,{'was_impossible': False},109,9,0.0
13168,1,3744,4.0,4.0,{'was_impossible': False},188,13,0.0
4713,177,344,4.0,4.0,{'was_impossible': False},725,127,0.0
4710,76,1288,4.0,4.0,{'was_impossible': False},94,53,0.0
4709,482,8665,4.0,4.0,{'was_impossible': False},104,64,0.0


### Graph Lab Model

In [66]:
# Make a train-test split
#train_data, validate_data = df.random_split(0.8)

### Cosine Similarity

In [67]:
from surprise import KNNBasic
from surprise.model_selection import cross_validate

In [68]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)

In [69]:
#knn.train(trainset)

### Surprise Baseline Only

In [70]:
from surprise import BaselineOnly

In [71]:
# showcasing the cross_validate function as well 
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1453  1.1517  1.1496  1.1416  1.1569  1.1490  0.0053  
MAE (testset)     0.8356  0.8401  0.8379  0.8315  0.8445  0.8379  0.0044  
Fit time          0.22    0.24    0.23    0.24    0.24    0.23    0.01    
Test time         0.08    0.23    0.08    0.08    0.08    0.11    0.06    


{'test_rmse': array([1.14530474, 1.15173813, 1.14956872, 1.14160906, 1.15690397]),
 'test_mae': array([0.83563609, 0.84008628, 0.83789109, 0.83148721, 0.8445311 ]),
 'fit_time': (0.2151339054107666,
  0.23809218406677246,
  0.23351311683654785,
  0.23587489128112793,
  0.23675107955932617),
 'test_time': (0.08026385307312012,
  0.23421382904052734,
  0.07959222793579102,
  0.08138012886047363,
  0.07633304595947266)}

### KNN with GridSearch

In [72]:
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV

In [73]:

sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [2, 3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

### Grid Search

In [74]:
# param_grid = {
        #"n_epochs": [5, 10, 15, 20, 30, 40, 50, 100],
        # "lr_all": [0.001, 0.002, 0.005],
       # "reg_all": [0.02, 0.08, 0.4, 0.6]
#}

# smaller grid for testing
param_grid = {
    "n_epochs": [10, 20],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.02]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], refit=True, cv=5)

gs.fit(data)

training_parameters = gs.best_params["rmse"]

print("BEST RMSE: \t", gs.best_score["rmse"])
print("BEST MAE: \t", gs.best_score["mae"])
print("BEST params: \t", gs.best_params["rmse"])

BEST RMSE: 	 1.1492473460940427
BEST MAE: 	 0.837505242306781
BEST params: 	 {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


### Movie Reccomendation Model

In [75]:
df_2

,movieId,imdbId,tmdbId,title,userId_x,rating,userId_y,tag,timestamp,new_genres
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Adventure
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Animation
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Children
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Comedy
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Fantasy
...,...,...,...,...,...,...,...,...,...,...
233211,193565,1636780,71172.0,Gintama: The Movie (2010),184,3.5,184,gintama,1537098603,Sci-Fi
233212,193565,1636780,71172.0,Gintama: The Movie (2010),184,3.5,184,remaster,1537098592,Action
233212,193565,1636780,71172.0,Gintama: The Movie (2010),184,3.5,184,remaster,1537098592,Animation
233212,193565,1636780,71172.0,Gintama: The Movie (2010),184,3.5,184,remaster,1537098592,Comedy


In [76]:
df_2.head()

,movieId,imdbId,tmdbId,title,userId_x,rating,userId_y,tag,timestamp,new_genres
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Adventure
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Animation
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Children
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Comedy
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Fantasy


In [77]:
import RAKE
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df_3 = df_2[['title','new_genres','tag','rating']]
df_3.head()

,title,new_genres,tag,rating
0,Toy Story (1995),Adventure,pixar,4.0
0,Toy Story (1995),Animation,pixar,4.0
0,Toy Story (1995),Children,pixar,4.0
0,Toy Story (1995),Comedy,pixar,4.0
0,Toy Story (1995),Fantasy,pixar,4.0


In [78]:
# initializing the new column
df_3['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['tag']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df_3.drop(columns = ['tag'], inplace = True)

<ipython-input-78-4cbf3a1506ab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['Key_words'] = ""


KeyError: 'tag'

In [79]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['tag'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

KeyError: 'tag'

In [ ]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [ ]:
recommendations('Toy Story (1995)')